In [1]:
from importlib import reload

import pandas as pd

import bookcrossing_dataset

In [4]:
reload(bookcrossing_dataset)
bookcrossing_dataset.process_dataset()

KeyError: 'ISBN'

In [3]:
items_df = bookcrossing_dataset.items_df()
with pd.option_context('display.max_colwidth', 1000):
    display(items_df)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,title,description,image_slug
item_id,,,,,,,,,,,
034545104X,034545104X,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.LZZZZZZZ.jpg,Flesh Tones: A Novel,None,[034545104X]
0155061224,0155061224,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0155061224.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0155061224.01.LZZZZZZZ.jpg,Rites of Passage,None,[0155061224]
0446520802,0446520802,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0446520802.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0446520802.01.LZZZZZZZ.jpg,The Notebook,None,[0446520802]
052165615X,052165615X,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/052165615X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/052165615X.01.LZZZZZZZ.jpg,Help!: Level 1,None,[052165615X]
0521795028,0521795028,The Amsterdam Connection : Level 4 (Cambridge English Readers),Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0521795028.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0521795028.01.LZZZZZZZ.jpg,The Amsterdam Connection : Level 4 (Cambridge English Readers),None,[0521795028]
...,...,...,...,...,...,...,...,...,...,...,...
0517145553,0517145553,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0517145553.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0517145553.01.LZZZZZZZ.jpg,Mostly Harmless,None,[0517145553]
1575660792,1575660792,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/1575660792.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/1575660792.01.LZZZZZZZ.jpg,Gray Matter,None,[1575660792]
0590907301,0590907301,Triplet Trouble and the Class Trip (Triplet Trouble),Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0590907301.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0590907301.01.LZZZZZZZ.jpg,Triplet Trouble and the Class Trip (Triplet Trouble),None,[0590907301]


In [5]:
reviews_df = bookcrossing_dataset.reviews_df()
reviews_df

,user_id,item_id,rating
0,276725,034545104X,0.0
1,276726,0155061224,2.5
2,276727,0446520802,0.0
3,276729,052165615X,1.5
4,276729,0521795028,3.0
...,...,...,...
1149774,276704,0876044011,0.0
1149775,276704,1563526298,4.5
1149776,276706,0679447156,0.0
1149777,276709,0515107662,5.0


In [6]:
with pd.option_context('display.max_colwidth', 1000):
    display(items_df.loc[items_df.index == '000200545X'])

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,title,description,image_slug
item_id,,,,,,,,,,,
000200545X,000200545X,Elegance,Kathleen Tessaro,2003,HarperCollins Publishers,http://images.amazon.com/images/P/000200545X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/000200545X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/000200545X.01.LZZZZZZZ.jpg,Elegance,None,[000200545X]


In [13]:
bookcrossing_dataset.download_images(items_df)

In [39]:
a =[
    (x.stem, x.stat().st_size >= 1000) for x in
    bookcrossing_dataset.IMAGE_FOLDER.glob('*.jpg')
]
has_image = pd.Series([x[1] for x in a], index=[x[0] for x in a])

In [40]:
items_df.loc[has_image]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,title,description,image_slug
item_id,,,,,,,,,,,
034545104X,034545104X,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,Flesh Tones: A Novel,None,[034545104X]
0155061224,0155061224,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,Rites of Passage,None,[0155061224]
0446520802,0446520802,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,The Notebook,None,[0446520802]
052165615X,052165615X,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,Help!: Level 1,None,[052165615X]
0521795028,0521795028,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,The Amsterdam Connection : Level 4 (Cambridge ...,None,[0521795028]
...,...,...,...,...,...,...,...,...,...,...,...
0312264186,0312264186,You Can't Catch Death: A Daughter's Memoir,Ianthe Brautigan,2001,St. Martin's Press,http://images.amazon.com/images/P/0312264186.0...,http://images.amazon.com/images/P/0312264186.0...,http://images.amazon.com/images/P/0312264186.0...,You Can't Catch Death: A Daughter's Memoir,None,[0312264186]
0425150526,0425150526,Death Crosses the Border,Janice Steinberg,1995,Berkley Publishing Group,http://images.amazon.com/images/P/0425150526.0...,http://images.amazon.com/images/P/0425150526.0...,http://images.amazon.com/images/P/0425150526.0...,Death Crosses the Border,None,[0425150526]
1575660792,1575660792,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,Gray Matter,None,[1575660792]


In [45]:
reviews_df.loc[reviews_df['item_id'].isin(items_df.loc[has_image].index)]

,user_id,item_id,rating
0,276725,034545104X,0.0
1,276726,0155061224,2.5
2,276727,0446520802,0.0
3,276729,052165615X,1.5
4,276729,0521795028,3.0
...,...,...,...
1149774,276704,0876044011,0.0
1149775,276704,1563526298,4.5
1149776,276706,0679447156,0.0
1149777,276709,0515107662,5.0
